In [1]:
import random
import math
#from Simple_Tabulation_Hash import Simple_Tabulation_Hash
#from Chave_Valor import Chave_Valor

In [2]:
class VEB:
    def get_c(self, x):
        return x >> int(self.w/2)

    def get_i(self, x):
        return x & ((1 << int(self.w/2)) - 1)

    def get_x(self, c, i):
        return (c << int(self.w/2)) | i

    def __init__(self, w):
        self.w = w
        self.min = None
        self.max = None

        self.clusters = [None for _ in range(w)]#Simple_Tabulation_Hash(8, w) #tamanho inicial da tabela = 8
        self.resumo = None

    def encontrar(self, x):
        if(x == self.min or x == self.max):
            return True
        elif(self.w <= 2):
            return False
        else:
            cluster = self.clusters[self.get_c(x)]
            if(cluster != None):
                return cluster.encontrar(self.get_i(x))
            else:
                return False

    def sucessor(self, x):
        if(self.w <= 2):
            if(x == 0 and self.max == 1):
                return 1
            else:
                return None
        elif(self.min != None and x < self.min):
            return self.min
        else:
            c = self.get_c(x)
            i = self.get_i(x)
            max_i = None
            cluster = self.clusters[c]
            if(cluster != None):
                max_i = cluster.max
            if(max_i != None and i < max_i):
                prox = cluster.sucessor(i)
                if(prox != None):
                    return self.get_x(c, prox)
            else:
                cluster_suc = None
                if(self.resumo != None):
                    cluster_suc = self.resumo.sucessor(c)
                if(cluster_suc == None):
                    return None
                else:
                    cluster_ = self.clusters[cluster_suc]
                    prox = 0
                    if(cluster_ != None):
                        prox = cluster_.min
                    return self.get_x(cluster_suc, prox)

    def predecessor(self, x):
        if(self.w <= 2):
            if(x == 1 and self.min == 0):
                return 0
            else:
                return None
        elif(self.max != None and x > self.max):
            return self.max
        else:
            c = self.get_c(x)
            i = self.get_i(x)
            min_i = None
            cluster = self.clusters[c]
            if(cluster != None):
                min_i = cluster.min
            if(min_i != None and i > min_i):
                ant = cluster.predecessor(i)
                if(ant == None): 
                    ant = 0
                return self.get_x(c, ant)
            else:
                cluster_pred = None
                if(self.resumo != None):
                    cluster_pred = self.resumo.predecessor(c)
                if(cluster_pred == None):
                    if(self.min != None and x > self.min):
                        return self.min
                    else:
                        return None
                else:
                    cluster_ = self.clusters[cluster_pred]
                    ant = 0
                    if(cluster_ != None):
                        ant = cluster_.max
                    return self.get_x(cluster_pred, ant)

    def incluir_unico(self, x):
        self.min = x
        self.max = x

    def incluir(self, x):
        if(not self.encontrar(x)):
            if(self.min == None):
                self.incluir_unico(x)
            else:
                if(x < self.min):
                    self.min, x = x, self.min
                if(self.w > 2):
                    c = self.get_c(x)
                    if(self.clusters[c] == None):
                        self.clusters[c] = VEB(int(self.w/2))
                    if(self.resumo == None):
                        self.resumo = VEB(int(self.w/2))
                    if(self.clusters[c].min == None):
                        self.resumo.incluir(c)
                        self.clusters[c].incluir_unico(self.get_i(x))
                    else:
                        self.clusters[c].incluir(self.get_i(x))
                if(x > self.max):
                    self.max = x
        else:
            return False
                
    def remover(self, x):
        if(self.encontrar(x)):
            if(x == self.min):
                if(self.resumo == None):
                    c = None
                else:
                    c = self.resumo.min
                if(c == None):
                    self.min = None
                    return True
                self.min = self.get_x(self.get_c(x), self.clusters[c].min)
                x = self.min
            c = self.get_c(x)
            self.clusters[c].remover(self.get_i(x))
            if(self.clusters[c].min == None):
                self.resumo.remover(c)
            if(self.resumo.min == None):
                self.max = self.min
            else:
                c_ = self.resumo.max
                self.max = self.get_x(c_, self.clusters[c_].max)
            return True
        else:
            return False
    
    def executar(self):
        arquivo = open('entrada.txt','r')
        saida = open('saida.txt','w')

        for line in arquivo:
            operacao, x = line.strip().split(':')
            if(operacao == 'INC'):
                #print(operacao,x)
                self.incluir(int(x))
                saida.write("{}:{}\n\n".format(operacao,x))
                
            elif(operacao == 'REM'):
                self.remover(int(x))
                saida.write("{}:{}\n\n".format(operacao,x))

            elif(operacao == 'SUC'):
                suc = self.sucessor(int(x))
                saida.write("{}:{}\n{}\n\n".format(operacao,x,suc))
                
            elif(operacao == 'PRE'):
                pred = self.predecessor(int(x))
                saida.write("{}:{}\n{}\n\n".format(operacao,x,pred))

        arquivo.close()
        saida.close()

In [3]:
veb = VEB(64)
veb.executar()

In [4]:
veb.incluir(1)

In [5]:
veb.incluir(2)

In [6]:
veb.incluir(3829)

In [7]:
veb.incluir(832)

In [8]:
veb.predecessor(3829)

832

In [9]:
veb.sucessor(833)

3829

In [10]:
veb.predecessor(2)

1

In [11]:
veb.sucessor(2)

10